### Movie for T, S, Tracers Tracers - Aids to visualize the flow

In [1]:
#KRM

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

import matplotlib as mpl
#%matplotlib inline
from math import *

import scipy.io

import scipy as spy

from netCDF4 import Dataset

import pylab as pl

import os 

import sys

import seaborn as sns

In [2]:
lib_path = os.path.abspath('../../Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 


In [9]:
CanyonGrid='/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/gridGlob.nc'
CanyonGridOut = Dataset(CanyonGrid)

CanyonState='/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/stateGlob.nc'
CanyonStateOut = Dataset(CanyonState)

CanyonPtracers='/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/ptracersGlob.nc'
CanyonPtracersOut = Dataset(CanyonPtracers)



In [14]:
#for dimobj in CanyonPtracersOut.variables.values():
#    print dimobj

In [11]:
# General input

nx = 360
ny = 360
nz = 90
nt = 19 # t dimension size 

z = CanyonStateOut.variables['Z']
Time = CanyonStateOut.variables['T']

xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells
yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells

bathy = rout.getField(CanyonGrid, 'Depth')

hFacC = rout.getField(CanyonGrid, 'HFacC')
MaskC = rout.getMask(CanyonGrid, 'HFacC')

hFacW = rout.getField(CanyonGrid, 'HFacW') # (Z,Y,Xp1) for zonal fluxes
MaskW = rout.getMask(CanyonGrid,'HFacW') 

hFacS = rout.getField(CanyonGrid, 'HFacS') # (Z,Yp1,X) for meridional fluxes
MaskS = rout.getMask(CanyonGrid,'HFacS') 


In [15]:
# Load velocity components
T = rout.getField(CanyonState,'Temp')
S = rout.getField(CanyonState,'S')
Tr01= rout.getField(CanyonPtracers,'Tr1')


In [16]:
print(np.max(T), np.min(T))
print(np.max(S), np.min(S))
print(np.max(Tr01), np.min(Tr01))

Tmax = np.max(T)
Smax = np.max(S)
Trmax=np.max(Tr01)

(20.0, 0.0)
(34.000996, 0.0)
(45.021442, 0.0)


In [17]:
import matplotlib.animation as animation

In [67]:
sns.set()
sns.set_style('white')
Tcmap = sns.cubehelix_palette(light=1, as_cmap=True)
Scmap = sns.light_palette("navy", as_cmap=True)
Trcmap = sns.cubehelix_palette(8, start=.5, rot=-.75, as_cmap=True)



In [113]:
def Plot1(t,ax1):
    
    Tplot=np.ma.array(T[t,:,:,xslice],mask=MaskC[:,:,xslice])
    csT = np.linspace(0.4,0.75,num=30)
    csT2 = np.linspace(0.4,0.75,num=15)
    ax1.clear()
    mesh=ax1.contourf(yc[100:,200],z[:58],Tplot[:58,100:]/Tmax,csT,cmap=Tcmap)
    if t == 1:    
        cbar=plt.colorbar(mesh,ax=ax1,cmap ="")
    CS = ax1.contour(yc[100:,200],z[:58],Tplot[:58,100:]/Tmax,csT2,colors='k',linewidths=[0.75] )
    ax1.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax1.set_xlabel('Across-shore distance [m]')
    ax1.set_ylabel('Depth [m]')
    ax1.set_title('Temperature at day %0.1f' %(t/2.0))
    plt.cla()

def Plot2(t,ax2):    
    Tplot=np.ma.array(T[t,:,yslice,:],mask=MaskC[:,yslice,:])
    csT = np.linspace(0.4,0.75,num=30)
    csT2 = np.linspace(0.4,0.75,num=15)
    ax2.clear()
    mesh=ax2.contourf(xc[200,100:260],z[:58],Tplot[:58,100:260]/Tmax,csT,cmap=Tcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax2)
    CS = ax2.contour(xc[200,100:260],z[:58],Tplot[:58,100:260]/Tmax,csT2,colors='k',linewidths=[0.75] )
    ax2.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax2.set_xlabel('Alongshore distance [m]')
    ax2.set_ylabel('Depth [m]')
    ax2.set_title('Temperature')
    
def Plot3(t,ax3):    
    Tplot = np.ma.array(T[t,zslice,:,:],mask=MaskC[zslice,:,:])
    csT = np.linspace(0.55,0.65,num=30)
    csT2 = np.linspace(0.55,0.65,num=15)
    ax3.clear()
    mesh=ax3.contourf(xc[150:250,:],yc[150:250,:],Tplot[150:250,:]/Tmax,csT,cmap=Tcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax3)
    CS = ax3.contour(xc[150:250,:],yc[150:250,:],bathy[150:250,:],[145],colors='k',linewidths=[2.0] )
    CSS = ax3.contour(xc[150:250,:],yc[150:250,:],Tplot[150:250,:]/Tmax,csT2,colors='k',linewidths=[0.75] )
    
    ax3.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax3.set_xlabel('Alongshore distance [m]')
    ax3.set_ylabel('Across-shore distance [m]')
    ax3.set_title('Temperature at shelf break')

def Plot4(t,ax4):
    Splot=np.ma.array(S[t,:,:,xslice],mask=MaskC[:,:,xslice])
    csS = np.linspace(0.94,0.97,num=30)
    csS2 = np.linspace(0.94,0.97,num=15)
    ax4.clear()
    
    mesh=ax4.contourf(yc[100:,200],z[:58],Splot[:58,100:]/Smax,csS,cmap=Scmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax4)
    CS=ax4.contour(yc[100:,200],z[:58],Splot[:58,100:]/Smax,csS2,colors='k',linewidths=[0.75])
    ax4.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax4.set_xlabel('Across-shore distance [m]')
    ax4.set_ylabel('Depth [m]')
    ax4.set_title('Salinity')
    
def Plot5(t,ax5):    
    Splot=np.ma.array(S[t,:,yslice,:],mask=MaskC[:,yslice,:])
    csS = np.linspace(0.94,0.97,num=30)
    csS2 = np.linspace(0.94,0.97,num=15)
    ax5.clear()
    mesh=ax5.contourf(xc[200,100:260],z[:58],Splot[:58,100:260]/Smax,csS,cmap=Scmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax5)
    CS=ax5.contour(xc[200,100:260],z[:58],Splot[:58,100:260]/Smax,csS2,colors='k',linewidths=[0.75])
    ax5.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax5.set_xlabel('Alongshore distance [m]')
    ax5.set_ylabel('Depth [m]')
    ax5.set_title('Salinity')

def Plot6(t,ax6):
    Splot = np.ma.array(S[t,zslice,:,:],mask=MaskC[zslice,:,:])
    csS = np.linspace(0.945,0.955,num=26)
    csS2 = np.linspace(0.945,0.955,num=13)
    ax6.clear()
    
    mesh=ax6.contourf(xc[150:250,:],yc[150:250,:],Splot[150:250,:]/Smax,csS,cmap=Scmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax6)
    CS = ax6.contour(xc[150:250,:],yc[150:250,:],bathy[150:250,:],[148],colors='k',linewidths=[2.25] )
    CSS=ax6.contour(xc[150:250,:],yc[150:250,:],Splot[150:250,:]/Smax,csS2,colors='k',linewidths=[0.75])
    
    ax6.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax6.set_xlabel('Alongshore distance [m]')
    ax6.set_ylabel('Across-shore distance [m]')
    ax6.set_title('Salinity at shelf break')

def Plot7(t,ax7):
    Trplot=np.ma.array(Tr01[t,:,:,xslice],mask=MaskC[:,:,xslice])
    csTr = np.linspace(0,0.5,num=30)
    csTr2 = np.linspace(0,0.5,num=15)
    ax7.clear()
    mesh=ax7.contourf(yc[100:,200],z[:58],Trplot[:58,100:]/Trmax,csTr,cmap=Trcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax7)
    CS=ax7.contour(yc[100:,200],z[:58],Trplot[:58,100:]/Trmax,csTr2,colors='k',linewidths=[0.75])
    ax7.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax7.set_xlabel('Across-shore distance [m]')
    ax7.set_ylabel('Depth [m]')
    ax7.set_title('Tracer concentration')

def Plot8(t,ax8):
    Trplot=np.ma.array(Tr01[t,:,yslice,:],mask=MaskC[:,yslice,:])
    csTr = np.linspace(0.0,0.5,num=30)
    csTr2 = np.linspace(0,0.5,num=15)
    ax8.clear()
    
    mesh=ax8.contourf(xc[200,100:260],z[:58],Trplot[:58,100:260]/Trmax,csTr,cmap=Trcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax8)
    CS=ax8.contour(xc[200,100:260],z[:58],Trplot[:58,100:260]/Trmax,csTr2,colors='k',linewidths=[0.75])
    ax8.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax8.set_xlabel('Alongshore distance [m]')
    ax8.set_ylabel('Depth [m]')
    ax8.set_title('Tracer concentration')
    
def Plot9(t,ax9):
    Trplot = np.ma.array(Tr01[t,zslice,:,:],mask=MaskC[zslice,:,:])
    csTr = np.linspace(0.14,0.24,num=30)
    csTr2 = np.linspace(0.14,0.24,num=15)
    ax9.clear()
    mesh=ax9.contourf(xc[150:250,:],yc[150:250,:],Trplot[150:250,:]/Trmax,csTr,cmap=Trcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax9)
    CS = ax9.contour(xc[150:250,:],yc[150:250,:],bathy[150:250,:],[145],colors='k',linewidths=[2.25] )
    CSS = ax9.contour(xc[150:250,:],yc[150:250,:],Trplot[150:250,:]/Trmax,csTr2,colors='k',linewidths=[0.75] )
    
    ax9.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax9.set_xlabel('Alongshore distance [m]')
    ax9.set_ylabel('Across-shore distance [m]')
    ax9.set_title('Tracer Concentration')

In [114]:
## Animation
#N=5
xslice=180
yslice=200
zslice= 29 # shelf break index
zslice2= 23 

#Empty figures
fig,((ax1,ax2,ax3),(ax4, ax5,ax6),(ax7,ax8,ax9))  = plt.subplots(3, 3, figsize=(20, 14))

#Initial image
def init():
    Plot1(0,ax1)
    Plot2(0,ax2)
    Plot3(0,ax3)
    Plot4(0,ax4)
    Plot5(0,ax5)
    Plot6(0,ax6)
    Plot7(0,ax7)
    Plot8(0,ax8)
    Plot9(0,ax9)
    
    plt.tight_layout()
    #return[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]
    

def animate(tt):
    Plot1(tt,ax1)
    Plot2(tt,ax2)
    Plot3(tt,ax3)
    Plot4(tt,ax4)
    Plot5(tt,ax5)
    Plot6(tt,ax6)
    Plot7(tt,ax7)
    Plot8(tt,ax8)
    Plot9(tt,ax9)
    plt.tight_layout()
#The animation function (max frames=47)   


anim = animation.FuncAnimation(fig, animate, init_func=init,frames=18, interval = 200,blit=False, repeat=False)

##A line that makes it all work
mywriter = animation.FFMpegWriter()

##Save in current folder
anim.save('Trac_GMREDI_FluxDiagnostics_run01.mp4',writer=mywriter,fps=0.5, dpi = 200, bitrate = 1000000, codec = "libx264")

plt.show()

